# **Importing OMIM database and checking it with the OMIM website data**

In [143]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Read the text file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/pardisgene/datasets/OMIM_genemap2_chr17.txt', sep='\t', comment='#')

In [146]:
# Function to determine inheritance based on the comment column
def determine_inheritance(comment: str) -> str:
    if isinstance(comment, str):  # Check if the comment is a string
        if 'autosomal dominant' in comment.lower():
            if 'autosomal recessive' in comment.lower():
                return 'AD/AR'
            else:
                return 'AD'
        elif 'autosomal recessive' in comment.lower():
            return 'AR'
        else:
            return 'Unknown'
    else:
        return 'Unknown'  # Return NaN for rows with NaN values in the comment column

# Add a new column for inheritance
df['MOI'] = df['Phenotypes'].apply(determine_inheritance)

In [147]:
df.head()

,Chromosome,Genomic Position Start,Genomic Position End,Cyto Location,Computed Cyto Location,MIM Number,Gene/Locus And Other Related Symbols,Gene Name,Approved Gene Symbol,Entrez Gene ID,Ensembl Gene ID,Comments,Phenotypes,Mouse Gene Symbol/ID,MOI
0,chr17,1,10800000,17p13,NaN,608631,ASPG2,"Asperger syndrome, susceptibility to, 2",NaN,431711.0,NaN,breakpoints between CHRNE and GP1BA,"{Asperger syndrome susceptibility 2}, 608631 (...",NaN,Unknown
1,chr17,1,3400000,17p13.3,NaN,113721,BCPR,Breast cancer-related regulator of TP53,NaN,NaN,NaN,NaN,Breast cancer (1),NaN,Unknown
2,chr17,1,10800000,17p13,NaN,601202,"CTRCT24, CTAA2","Cataract 24, anterior polar",NaN,1484.0,NaN,NaN,"Cataract 24, anterior polar, 601202 (2), Autos...",NaN,AD
3,chr17,1,3400000,17p13.3,NaN,615674,DDD3,Dowling-Degos disease 3,NaN,102997065.0,NaN,max lod at D17S1529,"Dowling-Degos disease 3, 615674 (2), Autosomal...",NaN,AD
4,chr17,1,3400000,17p13.3,NaN,613215,"DUP17p13.3, C17DUPp13.3",Chromosome 17p13.3 duplication syndrome,NaN,NaN,NaN,includes LIS1 and/or YWHAE,"Chromosome 17p13.3 duplication syndrome, 61321...",NaN,Unknown


In [148]:
df['MOI'].value_counts()

Unknown    756
AR         163
AD         114
AD/AR       34
Name: MOI, dtype: int64

In [149]:
!pip install omim

In [150]:
df['MIM Number'] = df['MIM Number'].astype(str)

In [151]:
import subprocess
import re
import omim
from tqdm import tqdm

def get_inheritance_mode(mim_number: str) -> str:
    """
    Retrieves the inheritance mode for a given MIM (Mendelian Inheritance in Man) number.

    Parameters:
        mim_number : str
            The MIM number for which the inheritance mode is to be retrieved.

    Returns:
        str
            The inheritance mode corresponding to the provided MIM number. Possible values are "AR" (Autosomal Recessive),
            "AD" (Autosomal Dominant), "AD/AR" (Autosomal Dominant and Recessive), "AR/AD" (Autosomal Recessive and Dominant),
            or "Unknown" if the inheritance mode could not be retrieved.

    This function utilizes the 'omim' command-line tool to query the OMIM database and extract the inheritance mode
    information for a given MIM number. It executes the command and captures the output, then extracts the inheritance
    mode using regular expressions. If the inheritance mode is not one of the predefined values ("AR", "AD", "AD/AR", "AR/AD"),
    it is marked as "Unknown". If an error occurs during the retrieval process, it returns "Error retrieving inheritance mode".

    """
    try:
        # Execute the command and capture its output
        output_bytes = subprocess.check_output(["omim", "query", "-s", "mim_number", mim_number])
        output = output_bytes.decode("utf-8")

        # Extract the inheritance mode from the output
        inheritance_mode_match = re.search(r'"Inheritance": "(\w+)"', output)
        if inheritance_mode_match:
            inheritance_mode = inheritance_mode_match.group(1)
            if inheritance_mode not in ['AR', 'AD', 'AD/AR', 'AR/AD']:
                inheritance_mode = "Unknown"
        else:
            inheritance_mode = "Unknown"
        return inheritance_mode
    except subprocess.CalledProcessError:
        return "Error retrieving inheritance mode"


# Iterate over rows of the DataFrame
for index, row in tqdm(df.iterrows()):
    # Check if the "inheritance2" column has the value "Unknown"
    if row['MOI'] == "Unknown":
        # Call the function to get the inheritance mode
        inheritance_mode = get_inheritance_mode(row['MIM Number'])
        # Update the "inheritance2" column with the retrieved inheritance mode
        df.at[index, 'MOI'] = inheritance_mode


1067it [10:45,  1.65it/s]


In [152]:
df['MOI'].value_counts()

Unknown    754
AR         163
AD         116
AD/AR       34
Name: MOI, dtype: int64

In [171]:
def add_new_data(df: pd.DataFrame , db_df: pd.DataFrame, gene_column_name: str, moi_column_name: str) -> None:
  """
  Updates the Mode of Inheritance (MOI) column in a DataFrame (df) based on information from another DataFrame (db_df).

  Parameters:
      df : pd.DataFrame
          The DataFrame containing the data to be updated.

      db_df : pd.DataFrame
          The DataFrame containing the reference data used for updating.

      gene_column_name : str
          The name of the column in db_df containing gene names.

      moi_column_name : str
          The name of the column in db_df containing Mode of Inheritance values.

  Returns:
      None

  This function iterates over each row in the input DataFrame (df). For each row where the Mode of Inheritance (MOI)
  value is 'Unknown', it extracts gene names from the 'Gene/Locus And Other Related Symbols' column and adds the 'Approved Gene Symbol'
  to the list if it's available. Then, it searches for each gene name in the reference DataFrame (db_df) using the specified
  gene column name. If a match is found, it updates the MOI value in the input DataFrame (df) with the corresponding value from
  the reference DataFrame (db_df). The search is performed based on the provided gene_column_name and moi_column_name.

  """

  for index, row in tqdm(df.iterrows()):
    # Check if the Inheritance value is 'Unknown'
    if row['MOI'] == 'Unknown':
        # Split the gene names if there are multiple names
        gene_names = row['Gene/Locus And Other Related Symbols'].split(',')

        # Add HGNC to gene_names list
        if type(row['Approved Gene Symbol']) == str and row['Approved Gene Symbol'] not in gene_names:
            gene_names.append(row['Approved Gene Symbol'])

        # Iterate over each gene name
        for gene_name in gene_names:
            # Search for the gene in db_df
            matching_row = db_df[db_df[gene_column_name] == gene_name.strip()]  # Strip to remove any leading/trailing spaces
            if not matching_row.empty:
                # Update Inheritance column in df with the value from db_df
                if matching_row[moi_column_name].values[0].upper() in ['AR', 'AD', 'AD/AR', 'AR/AD']:
                    df.at[index, 'MOI'] = matching_row[moi_column_name].values[0].upper()
                break  # Break out of the loop if a match is found for any gene name

# **Importing CGD database and concating with the OMIM database**

In [155]:
cgd_db = pd.read_excel('/content/drive/MyDrive/pardisgene/datasets/CGD all.xls')
cgd_db.head()

,GENE,HGNC ID,ENTREZ GENE ID,CONDITION,INHERITANCE,AGE GROUP,ALLELIC CONDITIONS,MANIFESTATION CATEGORIES,INTERVENTION CATEGORIES,COMMENTS,INTERVENTION/RATIONALE,REFERENCES
0,A2M,7,2,Alpha-2-macroglobulin deficiency,AD,NaN,NaN,General,General,Variants have been implicated in pulmonary dis...,The clinical consequences of variants are unclear,94459; 2475424; 1370808
1,A2ML1,23336,144568,"Otitis media, susceptibility to",AD,Pediatric,NaN,Allergy/Immunology/Infectious,Allergy/Immunology/Infectious,NaN,Individuals may have increased susceptibility ...,26121085
2,A4GALT,18149,53947,"Blood group, P1PK system",BG,Pediatric,NaN,Hematologic,Hematologic,NaN,Variants associated with a blood group may be ...,10993874
3,AAAS,13666,8086,Achalasia-addisonianism-alacrimia syndrome,AR,Pediatric,NaN,Dermatologic; Endocrine; Gastrointestinal; Neu...,Endocrine,NaN,Surveillance and treatment/preventive measures...,78049; 6243664; 3565479; 1537368; 8006362; 789...
4,AAGAB,25662,79719,"Keratoderma, palmoplantar, punctate type IA",AD,NaN,NaN,Dermatologic,General,NaN,Genetic knowledge may be beneficial related to...,23000146; 23064416


In [156]:
cgd_db['INHERITANCE'].value_counts()

AR                                                                                                    2470
AD                                                                                                    1378
AD/AR                                                                                                  439
XL                                                                                                     249
BG                                                                                                      31
Maternal                                                                                                28
AD/AR/Digenic                                                                                           21
AR/Digenic                                                                                              17
AD/Digenic                                                                                              13
Digenic                              

In [157]:
add_new_data(df, cgd_db, 'GENE', 'INHERITANCE')

1067it [00:01, 784.48it/s]


In [158]:
df['MOI'].value_counts()

Unknown    738
AR         168
AD         126
AD/AR       35
Name: MOI, dtype: int64

# **Importing Clingen database and concating with the OMIM, CGD database**

In [159]:
clingen_db = pd.read_csv("/content/drive/MyDrive/pardisgene/datasets/Clingen-Gene-Disease-Summary-2024-02-12.csv", sep=',', comment='#')

In [160]:
clingen_db.head()

,GENE SYMBOL,GENE ID (HGNC),DISEASE LABEL,DISEASE ID (MONDO),MOI,SOP,CLASSIFICATION,ONLINE REPORT,CLASSIFICATION DATE,GCEP
0,A2ML1,HGNC:23336,Noonan syndrome,MONDO:0018997,AD,SOP5,Disputed,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T16:00:00.000Z,RASopathy
1,AARS2,HGNC:21022,mitochondrial disease,MONDO:0044970,AR,SOP8,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2022-04-18T16:00:00.000Z,Mitochondrial Diseases
2,AASS,HGNC:17366,hyperlysinemia,MONDO:0009388,AR,SOP9,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2022-10-14T16:00:00.000Z,Aminoacidopathy
3,ABAT,HGNC:23,developmental and epileptic encephalopathy,MONDO:0100062,AR,SOP8,Moderate,https://search.clinicalgenome.org/kb/gene-vali...,2022-04-19T16:00:00.000Z,Epilepsy
4,ABCA4,HGNC:34,ABCA4-related retinopathy,MONDO:0800406,AR,SOP9,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2022-10-06T16:00:00.000Z,Retina


In [161]:
clingen_db['MOI'].value_counts()

AR              1155
AD              1067
XL               164
MT                47
SD                41
Undetermined      15
Name: MOI, dtype: int64

In [162]:
add_new_data(df, clingen_db, 'GENE SYMBOL', 'MOI')

1067it [00:00, 1146.40it/s]


In [163]:
df['MOI'].value_counts()

Unknown    731
AR         169
AD         132
AD/AR       35
Name: MOI, dtype: int64

# **Importing GENCC database and concating with the existing database**

In [164]:
gencc_db = pd.read_excel("/content/drive/MyDrive/pardisgene/datasets/gencc-submissions.xlsx")

In [165]:
gencc_db.head()

,uuid,gene_curie,gene_symbol,disease_curie,disease_title,disease_original_curie,disease_original_title,classification_curie,classification_title,moi_curie,...,submitted_as_submitter_name,submitted_as_classification_id,submitted_as_classification_name,submitted_as_date,submitted_as_public_report_url,submitted_as_notes,submitted_as_pmids,submitted_as_assertion_criteria_url,submitted_as_submission_id,submitted_run_date
0,GENCC_000101-HGNC_10896-OMIM_182212-HP_0000006...,HGNC:10896,SKI,MONDO:0008426,Shprintzen-Goldberg syndrome,OMIM:182212,Shprintzen-Goldberg syndrome,GENCC:100001,Definitive,HP:0000006,...,Ambry Genetics,GENCC:100001,Definitive,2018-03-30 13:31:56,NaN,NaN,NaN,PMID: 28106320,1034,2020-12-24
1,GENCC_000101-HGNC_16636-OMIM_171300-HP_0000006...,HGNC:16636,KIF1B,MONDO:0008233,pheochromocytoma,OMIM:171300,"{Pheochromocytoma, susceptibility to}",GENCC:100003,Moderate,HP:0000006,...,Ambry Genetics,GENCC:100003,Moderate,2019-12-04 13:30:43,NaN,NaN,NaN,PMID: 28106320,69237,2020-12-24
2,GENCC_000101-HGNC_16636-OMIM_118210-HP_0000006...,HGNC:16636,KIF1B,MONDO:0007308,Charcot-Marie-Tooth disease type 2A1,OMIM:118210,"Charcot-Marie-Tooth disease, type 2A1",GENCC:100004,Limited,HP:0000006,...,Ambry Genetics,GENCC:100004,Limited,2022-09-02 00:00:00,NaN,NaN,NaN,https://onlinelibrary.wiley.com/doi/10.1002/hu...,61327,2023-08-19
3,GENCC_000101-HGNC_17939-OMIM_617532-HP_0000007...,HGNC:17939,SLC45A1,MONDO:0044322,intellectual developmental disorder with neuro...,OMIM:617532,Intellectual developmental disorder with neuro...,GENCC:100004,Limited,HP:0000007,...,Ambry Genetics,GENCC:100004,Limited,2020-06-26 13:32:00,NaN,NaN,NaN,PMID: 28106320,72178,2020-12-24
4,GENCC_000101-HGNC_11071-OMIM_616291-HP_0000007...,HGNC:11071,SLC9A1,MONDO:0014572,Lichtenstein-Knorr syndrome,OMIM:616291,Lichtenstein-Knorr syndrome,GENCC:100004,Limited,HP:0000007,...,Ambry Genetics,GENCC:100004,Limited,2018-08-31 13:32:02,NaN,NaN,NaN,PMID: 28106320,1705,2020-12-24


In [166]:
gencc_db['moi_title'].value_counts()

Autosomal recessive                                                   10477
Autosomal dominant                                                     8910
X-linked                                                               1095
Unknown                                                                 892
X-linked recessive                                                      144
Semidominant                                                            133
Mitochondrial                                                           100
X-linked dominant                                                        26
Somatic mosaicism                                                         9
Autosomal dominant inheritance with paternal imprinting                   5
Autosomal dominant inheritance with maternal imprinting HP:0012275        4
Y-linked inheritance                                                      2
Digenic inheritance HP:0010984                                            1
Name: moi_ti

In [167]:
mapping = {
    'Autosomal dominant': 'AD',
    'Autosomal recessive': 'AR',
}


# Replace values in the 'moi_title' column
gencc_db['moi_title'] = gencc_db['moi_title'].map(lambda x: mapping.get(x, 'not found'))

In [169]:
add_new_data(df, gencc_db, 'gene_symbol', 'moi_title')

1067it [00:05, 203.79it/s]


In [170]:
df['MOI'].value_counts()

Unknown    716
AR         175
AD         141
AD/AR       35
Name: MOI, dtype: int64

# **Loading P_ADs and adding them to df**

In [172]:
import os

p_ad_data_list = []

# Initialize an empty DataFrame to store the data
p_ad_df = pd.DataFrame(columns=['HGNC', 'p_ad'])

# Loop through each text file
for i in range(100, 1001, 100):
    filename = f"{i}.txt"
    filepath = os.path.join("/content/drive/MyDrive/pardisgene/P_AD/", filename)  # Update the path to your files
    with open(filepath, 'r') as file:
        lines = file.readlines()
        # Iterate through the lines in the file
        for j in range(0, len(lines), 6):
            # Extract the first and fifth lines of each section
            gene_name = lines[j].strip()
            score = float(lines[j + 4].strip())
            # Append the data to the DataFrame
            p_ad_data_list.append({'HGNC': gene_name, 'p_ad': score})

# Print the DataFrame
p_ad_df = pd.concat([p_ad_df, pd.DataFrame(p_ad_data_list)], ignore_index=True)


In [173]:
p_ad_df.head()

,HGNC,p_ad
0,AANAT,0.068
1,AARSD1,0.099
2,AATF,0.097
3,AATK,0.205
4,ABCA10,0.125


In [174]:
# Merge the two DataFrames based on 'HGNC' and 'Approved Gene Symbol' columns
merged_df = pd.merge(df, p_ad_df, left_on='Approved Gene Symbol', right_on='HGNC', how='left')

# Drop the 'HGNC' column from the merged DataFrame
merged_df.drop(columns=['HGNC'], inplace=True)

In [175]:
merged_df.head()

,Chromosome,Genomic Position Start,Genomic Position End,Cyto Location,Computed Cyto Location,MIM Number,Gene/Locus And Other Related Symbols,Gene Name,Approved Gene Symbol,Entrez Gene ID,Ensembl Gene ID,Comments,Phenotypes,Mouse Gene Symbol/ID,MOI,p_ad
0,chr17,1,10800000,17p13,NaN,608631,ASPG2,"Asperger syndrome, susceptibility to, 2",NaN,431711.0,NaN,breakpoints between CHRNE and GP1BA,"{Asperger syndrome susceptibility 2}, 608631 (...",NaN,Unknown,NaN
1,chr17,1,3400000,17p13.3,NaN,113721,BCPR,Breast cancer-related regulator of TP53,NaN,NaN,NaN,NaN,Breast cancer (1),NaN,Unknown,NaN
2,chr17,1,10800000,17p13,NaN,601202,"CTRCT24, CTAA2","Cataract 24, anterior polar",NaN,1484.0,NaN,NaN,"Cataract 24, anterior polar, 601202 (2), Autos...",NaN,AD,NaN
3,chr17,1,3400000,17p13.3,NaN,615674,DDD3,Dowling-Degos disease 3,NaN,102997065.0,NaN,max lod at D17S1529,"Dowling-Degos disease 3, 615674 (2), Autosomal...",NaN,AD,NaN
4,chr17,1,3400000,17p13.3,NaN,613215,"DUP17p13.3, C17DUPp13.3",Chromosome 17p13.3 duplication syndrome,NaN,NaN,NaN,includes LIS1 and/or YWHAE,"Chromosome 17p13.3 duplication syndrome, 61321...",NaN,Unknown,NaN


In [176]:
print(f'number of NAN in p_ad column: {merged_df["p_ad"].isna().sum()}')
print(f'number of values in p_ad column: {merged_df["p_ad"].notna().sum()}')

number of NAN in p_ad column: 126
number of values in p_ad column: 941


In [177]:
merged_df.to_csv('/content/drive/MyDrive/pardisgene/datasets/final_df.csv', index=False)

# **Starting second part**

In [3]:
def generate_final_output(df : pd.DataFrame, vcf_df: pd.DataFrame) -> pd.DataFrame:
  """
    Generates a final output DataFrame based on matching criteria between two input DataFrames.

    Parameters:
        df : pd.DataFrame
            DataFrame containing genomic position information and associated data.
        vcf_df: pd.DataFrame
            DataFrame representing variant call data.

    Returns:
        pd.DataFrame
            Final output DataFrame containing merged information.

    This function takes two DataFrames as input: 'df' containing genomic position information
    and associated metadata, and 'vcf_df' containing variant call data. It iterates through each
    row of 'vcf_df' and identifies matching rows in 'df' based on genomic position. For each match,
    it constructs a new data dictionary containing relevant information from both DataFrames,
    including the variant ID, associated gene symbol, mode of inheritance, and probability of
    association (P_AD). If no match is found for a row in 'vcf_df', the corresponding entry in
    the output DataFrame contains null values. The function then concatenates all new data
    dictionaries into a final output DataFrame and returns it.
    """
  vcf_df_output = pd.DataFrame(columns=['ID', 'SYMBOL', 'MOI', 'P_AD'])

  new_data_list = []

  # Iterate over each row in vcf_df
  for _, row in vcf_df.iterrows():
      # Get the POS value from vcf_df
      pos = row['POS']

      # Check if pos falls within the range defined by 'Genomic Position Start' and 'Genomic Position End' in df
      matching_rows = df[(df['Genomic Position Start'] <= pos) & (df['Genomic Position End'] >= pos)]

      # Check if there are multiple matching rows in df
      if not matching_rows.empty:
          for _, matching_row in matching_rows.iterrows():
              new_data = {'ID': row['ID'],
                          'SYMBOL': matching_row['Gene/Locus And Other Related Symbols'],
                          # 'SYMBOL': matching_row['Approved Gene Symbol'],
                          'MOI': matching_row['MOI'],
                          'P_AD': matching_row['p_ad']}

              new_data_list.append(new_data)
      else:
          new_data = {'ID': row['ID'],
                      'SYMBOL': None,
                      'MOI': None,
                      'P_AD': None}

          new_data_list.append(new_data)


  vcf_df_output = pd.concat([vcf_df_output, pd.DataFrame(new_data_list)], ignore_index=True)

  return vcf_df_output

In [132]:
from pathlib import Path

df = pd.read_csv('/content/drive/MyDrive/pardisgene/datasets/final_df.csv')

# Define the directory containing the VCF files
vcf_directory = Path('/content/drive/MyDrive/pardisgene/vcf')  # Update with the path to your directory

# Define the directory to save the outputs
output_directory = Path('/content/drive/MyDrive/pardisgene/outputs')  # Update with the path to your output directory

# Iterate over each VCF file in the directory
for vcf_file in vcf_directory.glob('SMP*.hg38.vcf'):
    # Read the VCF file
    if str(vcf_file) in ['/content/drive/MyDrive/pardisgene/vcf/SMPERA1989.hg38.vcf',
                    '/content/drive/MyDrive/pardisgene/vcf/SMPERA2052.hg38.vcf']:

      vcf_df = pd.read_csv(vcf_file , sep='\t', skiprows=23)

    else:
      vcf_df = pd.read_csv(vcf_file , sep='\t', skiprows=24)


    try:
      final_output_df = generate_final_output(df, vcf_df)

      # Save the processed data to the output directory
      output_file_path = f'{output_directory}/{vcf_file.stem.split(".")[0]}.csv'
      final_output_df.to_csv(output_file_path, index=False)


      print(f"Processed file {vcf_file} saved as {output_file_path}")
    except Exception as e:
      print(f"Error processing file {vcf_file}: {e}")

Processed file /content/drive/MyDrive/pardisgene/vcf/SMPERA1179.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPERA1179.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPERA1362.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPERA1362.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPERA1441.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPERA1441.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPQRA1567.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPQRA1567.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPQRA1379.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPQRA1379.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPERA1559.hg38.vcf saved as /content/drive/MyDrive/pardisgene/outputs/non_match/SMPERA1559.csv
Processed file /content/drive/MyDrive/pardisgene/vcf/SMPERC1501.hg38.vcf saved as /content/drive/MyDrive/p

In [122]:
final_output_df.value_counts('MOI')

MOI
Unknown    8293
AD         3485
AR         1563
AD/AR       110
dtype: int64